In [1]:
import nasdaqdatalink
data = nasdaqdatalink.get('OPT/SPX')

In [5]:
import numpy as np
import pandas as pd

In [37]:
row=data.iloc[0].to_dict()
row
out_terms=np.array([1,30,60.90,120,540])

In [38]:

#row=data[0].to_dict()
row=data.iloc[0].to_dict()
#compute vols

terms={'iv30':30,
       'iv60':60,
       'iv90':90,
}
for i in range(1,5):
    terms['m'+str(i)+'atmiv']=row["m"+str(i)+"dtex"]

sorted_arr=sorted(terms.items(), key=lambda x: x[1])
terms=np.array([x[1] for x in sorted_arr])
vols=np.array([row[x[0]] for x in sorted_arr])

weights30=np.sqrt(30)/np.sqrt(np.maximum(terms,30))
weights_inf=1-weights30
orats_interp_slopes=weights30*row["slope"]+(1-weights30)*row["slope_inf"]
orats_interp_derivs=weights30*row["deriv"]+(1-weights30)*row["deriv_inf"]
texp=terms/365.25

terms
orats_interp_slopes

import py_vollib.black_scholes
import py_vollib_vectorized

option_price90=py_vollib_vectorized.models.vectorized_black_scholes_merton('c', 100,95,texp, 0, vols,0,return_as='array')
option_greeks90 = py_vollib_vectorized.get_all_greeks('c', 100,95,texp, 0, vols,0,model='black_scholes_merton', return_as='dict')
option_price110=py_vollib_vectorized.models.vectorized_black_scholes_merton('c', 100,105,texp, 0, vols,0,return_as='array')
option_greeks110 = py_vollib_vectorized.get_all_greeks('c', 100,105,texp, 0, vols/100,0,model='black_scholes_merton', return_as='dict')
delta90=np.array(option_greeks90['delta'])
delta110=np.array(option_greeks110['delta'])
vols
vols90=vols*(1+(orats_interp_slopes/1000+(orats_interp_derivs/1000*(delta90*100-50)/2))*(delta90*100-50))
vols110=vols*(1+(orats_interp_slopes/1000+(orats_interp_derivs/1000*(delta110*100-50)/2))*(delta110*100-50))
log_slopes=(vols110-vols90)/(np.log(1.05/.95)*10)*np.sqrt(texp)
log_slopes
out_slopes=np.interp(out_terms,terms,log_slopes)
out_vols=np.interp(out_terms,terms,vols)

array([-1.49186414, -2.58679762, -4.08276237, -5.64073486, -5.64073486])